## 载入Python包

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
import mlxtend
from mlxtend.regressor import StackingRegressor
from mlxtend.regressor import StackingCVRegressor
import warnings
import gc
import sys
from pickle import dump
from pickle import load
import time
from xgboost import XGBRegressor
from tqdm import tqdm_notebook
from itertools import product
from tqdm import tqdm_notebook

warnings.filterwarnings('ignore')
pd.set_option('display.max_rows',500)
pd.set_option('display.max_columns',100)

%matplotlib inline

## 载入数据并预览

In [2]:
train = pd.read_csv('sales_train_v2.csv')
shops = pd.read_csv('shops.csv')
items = pd.read_csv('items.csv')
cats = pd.read_csv('item_categories.csv')
test = pd.read_csv('test.csv')

In [ ]:
train.head()

In [ ]:
shops.head()

In [ ]:
items.head()

In [ ]:
cats.head()

In [ ]:
test.head()

## 数据预处理

时间格式化

In [ ]:
train['date'] = pd.to_datetime(train.date,format="%d.%m.%Y")
train.head()

异常值检测,绘制箱型图

In [ ]:
train[['shop_id','item_id','item_price','item_cnt_day']].plot(kind='box',subplots=True,layout=(2,2),sharex=False,sharey=False)

选取需要剔除的偏离值

In [ ]:
train = train[train['item_price']<100000]
train = train[train['item_cnt_day']<1001]

对销量和价格小于0的值进行替换

In [ ]:
train.loc[train.item_price<0, 'item_price'] = 0
train.loc[train.item_cnt_day<0,'item_cnt_day'] = 0

## Shops/Items/Cats的特征预处理

Shops特征预处理

In [ ]:
shops['city'] = shops['shop_name'].str.split(' ').map(lambda x:x[0])
shops['city_code'] = LabelEncoder().fit_transform(shops['city'])
shops = shops[['shop_id','city_code']]

In [ ]:
shops.head()

Cats的特征预处理

In [ ]:
cats['type'] = cats['item_category_name'].str.split('-').map(lambda x:x[0].strip())
cats['type_code'] = LabelEncoder().fit_transform(cats['type'])
cats['subtype'] = cats['item_category_name'].str.split('-').map(lambda x:x[1].strip() if len(x)>1 else x[0].strip())
cats['subtype_code'] = LabelEncoder().fit_transform(cats['subtype'])
cats = cats[['item_category_id','type_code','subtype_code']]

In [ ]:
cats.head()

Items的特征预处理

In [ ]:
items.drop(['item_name'],axis=1,inplace=True)
items.head()

## 生成月销量特征

In [ ]:
start = time.time()
matrix = []
col = ['date_block_num','shop_id','item_id']
for i in range(34):
    sales = train[train['date_block_num']==i]
    matrix.append(np.array(list(product([i],sales['shop_id'].unique(),sales['item_id'].unique())),dtype='int16'))

matrix = pd.DataFrame(np.vstack(matrix),columns=col)
matrix['date_block_num'] = matrix['date_block_num'].astype(np.int8)
matrix['shop_id'] = matrix['shop_id'].astype(np.int8)
matrix.sort_values(col,inplace=True)
time.time()-start

In [ ]:
train['revenue'] = train['item_price']*train['item_cnt_day']
group = train.groupby(['date_block_num','shop_id','item_id']).agg({'item_cnt_day':'sum'})
group.columns = ['item_cnt_month']
group.reset_index(inplace=True)
matrix = pd.merge(matrix,group,on=col,how='left')
matrix['item_cnt_month'] = matrix['item_cnt_month'].fillna(0).clip(0.0,20).astype(np.float16)

In [ ]:
matrix.head()

## 连接测试集

In [ ]:
test['date_block_num'] = 34
test['date_block_num'] = test['date_block_num'].astype(np.int8)
test['shop_id'] = test['shop_id'].astype(np.int8)
test['item_id'] = test['item_id'].astype(np.int16)

matrix = pd.concat([matrix,test],keys=col,ignore_index=True).fillna(0)
matrix['ID'] = matrix['ID'].astype(np.int8)

In [ ]:
matrix.head()

## Shops/Items/Cats特征连接

In [ ]:
matrix = pd.merge(matrix, shops, on=['shop_id'], how='left')
matrix = pd.merge(matrix, items, on=['item_id'], how='left')
matrix = pd.merge(matrix, cats, on=['item_category_id'], how='left')

matrix['city_code'] = matrix['city_code'].astype(np.int8)
matrix['item_category_id'] = matrix['item_category_id'].astype(np.int8)
matrix['type_code'] = matrix['type_code'].astype(np.int8)
matrix['subtype_code'] = matrix['subtype_code'].astype(np.int8)

In [ ]:
matrix.head()

## 生成滞后特征

In [ ]:
def lag_feature(df, lags, col):
    tmp = df[['date_block_num','shop_id','item_id',col]]
    for i in lags:
        shifted = tmp.copy()
        shifted.columns = ['date_block_num','shop_id','item_id', col+'_lag_'+str(i)]
        shifted['date_block_num'] += i
        df = pd.merge(df, shifted, on=['date_block_num','shop_id','item_id'], how='left').fillna(0)
    return df

In [ ]:
matrix = lag_feature(matrix, [1,2,3,6,12], 'item_cnt_month')

## 生成均值特征

In [ ]:
group = matrix.groupby(['date_block_num']).agg({'item_cnt_month': ['mean']})
group.columns = [ 'date_avg_item_cnt' ]
group.reset_index(inplace=True)

In [ ]:
group.head()

In [ ]:
matrix = pd.merge(matrix, group, on=['date_block_num'], how='left')
matrix['date_avg_item_cnt'] = matrix['date_avg_item_cnt'].astype(np.float16)
matrix = lag_feature(matrix, [1], 'date_avg_item_cnt')
matrix.drop(['date_avg_item_cnt'], axis=1, inplace=True)

In [ ]:
group = matrix.groupby(['date_block_num', 'item_id']).agg({'item_cnt_month': ['mean']})
group.columns = [ 'date_item_avg_item_cnt' ]
group.reset_index(inplace=True)

matrix = pd.merge(matrix, group, on=['date_block_num','item_id'], how='left')
matrix['date_item_avg_item_cnt'] = matrix['date_item_avg_item_cnt'].astype(np.float16)
matrix = lag_feature(matrix, [1,2,3,6,12], 'date_item_avg_item_cnt')
matrix.drop(['date_item_avg_item_cnt'], axis=1, inplace=True)

In [ ]:
group = matrix.groupby(['date_block_num', 'shop_id']).agg({'item_cnt_month': ['mean']})
group.columns = [ 'date_shop_avg_item_cnt' ]
group.reset_index(inplace=True)

matrix = pd.merge(matrix, group, on=['date_block_num','shop_id'], how='left')
matrix['date_shop_avg_item_cnt'] = matrix['date_shop_avg_item_cnt'].astype(np.float16)
matrix = lag_feature(matrix, [1,2,3,6,12], 'date_shop_avg_item_cnt')
matrix.drop(['date_shop_avg_item_cnt'], axis=1, inplace=True)

In [ ]:
group = matrix.groupby(['date_block_num', 'item_category_id']).agg({'item_cnt_month': ['mean']})
group.columns = [ 'date_cat_avg_item_cnt' ]
group.reset_index(inplace=True)

matrix = pd.merge(matrix, group, on=['date_block_num','item_category_id'], how='left')
matrix['date_cat_avg_item_cnt'] = matrix['date_cat_avg_item_cnt'].astype(np.float16)
matrix = lag_feature(matrix, [1], 'date_cat_avg_item_cnt')
matrix.drop(['date_cat_avg_item_cnt'], axis=1, inplace=True)

In [ ]:
group = matrix.groupby(['date_block_num', 'shop_id', 'item_category_id']).agg({'item_cnt_month': ['mean']})
group.columns = ['date_shop_cat_avg_item_cnt']
group.reset_index(inplace=True)

matrix = pd.merge(matrix, group, on=['date_block_num', 'shop_id', 'item_category_id'], how='left')
matrix['date_shop_cat_avg_item_cnt'] = matrix['date_shop_cat_avg_item_cnt'].astype(np.float16)
matrix = lag_feature(matrix, [1], 'date_shop_cat_avg_item_cnt')
matrix.drop(['date_shop_cat_avg_item_cnt'], axis=1, inplace=True)

In [ ]:
group = matrix.groupby(['date_block_num', 'shop_id', 'type_code']).agg({'item_cnt_month': ['mean']})
group.columns = ['date_shop_type_avg_item_cnt']
group.reset_index(inplace=True)

matrix = pd.merge(matrix, group, on=['date_block_num', 'shop_id', 'type_code'], how='left')
matrix['date_shop_type_avg_item_cnt'] = matrix['date_shop_type_avg_item_cnt'].astype(np.float16)
matrix = lag_feature(matrix, [1], 'date_shop_type_avg_item_cnt')
matrix.drop(['date_shop_type_avg_item_cnt'], axis=1, inplace=True)

In [ ]:
group = matrix.groupby(['date_block_num', 'city_code']).agg({'item_cnt_month': ['mean']})
group.columns = [ 'date_city_avg_item_cnt' ]
group.reset_index(inplace=True)

matrix = pd.merge(matrix, group, on=['date_block_num', 'city_code'], how='left')
matrix['date_city_avg_item_cnt'] = matrix['date_city_avg_item_cnt'].astype(np.float16)
matrix = lag_feature(matrix, [1], 'date_city_avg_item_cnt')
matrix.drop(['date_city_avg_item_cnt'], axis=1, inplace=True)

In [ ]:
group = matrix.groupby(['date_block_num', 'item_id', 'city_code']).agg({'item_cnt_month': ['mean']})
group.columns = [ 'date_item_city_avg_item_cnt' ]
group.reset_index(inplace=True)

matrix = pd.merge(matrix, group, on=['date_block_num', 'item_id', 'city_code'], how='left')
matrix['date_item_city_avg_item_cnt'] = matrix['date_item_city_avg_item_cnt'].astype(np.float16)
matrix = lag_feature(matrix, [1], 'date_item_city_avg_item_cnt')
matrix.drop(['date_item_city_avg_item_cnt'], axis=1, inplace=True)

In [ ]:
group = matrix.groupby(['date_block_num', 'type_code']).agg({'item_cnt_month': ['mean']})
group.columns = [ 'date_type_avg_item_cnt' ]
group.reset_index(inplace=True)

matrix = pd.merge(matrix, group, on=['date_block_num', 'type_code'], how='left')
matrix['date_type_avg_item_cnt'] = matrix['date_type_avg_item_cnt'].astype(np.float16)
matrix = lag_feature(matrix, [1], 'date_type_avg_item_cnt')
matrix.drop(['date_type_avg_item_cnt'], axis=1, inplace=True)

In [ ]:
group = matrix.groupby(['date_block_num', 'subtype_code']).agg({'item_cnt_month': ['mean']})
group.columns = [ 'date_subtype_avg_item_cnt' ]
group.reset_index(inplace=True)

matrix = pd.merge(matrix, group, on=['date_block_num', 'subtype_code'], how='left')
matrix['date_subtype_avg_item_cnt'] = matrix['date_subtype_avg_item_cnt'].astype(np.float16)
matrix = lag_feature(matrix, [1], 'date_subtype_avg_item_cnt')
matrix.drop(['date_subtype_avg_item_cnt'], axis=1, inplace=True)

## 趋势特征

In [ ]:
train.head()

In [ ]:
group = train.groupby(['item_id']).agg({'item_price': ['mean']})
group.columns = ['item_avg_item_price']
group.reset_index(inplace=True)

matrix = pd.merge(matrix, group, on=['item_id'], how='left')
matrix['item_avg_item_price'] = matrix['item_avg_item_price'].astype(np.float16)

group = train.groupby(['date_block_num','item_id']).agg({'item_price': ['mean']})
group.columns = ['date_item_avg_item_price']
group.reset_index(inplace=True)

matrix = pd.merge(matrix, group, on=['date_block_num','item_id'], how='left')
matrix['date_item_avg_item_price'] = matrix['date_item_avg_item_price'].astype(np.float16)

lags = [1,2,3,4,5,6]
matrix = lag_feature(matrix, lags, 'date_item_avg_item_price')

In [ ]:
for i in lags:
    matrix['delta_price_lag_'+str(i)] = \
        (matrix['date_item_avg_item_price_lag_'+str(i)] - matrix['item_avg_item_price']) / matrix['item_avg_item_price']

def select_trend(row):
    for i in lags:
        if row['delta_price_lag_'+str(i)]:
            return row['delta_price_lag_'+str(i)]
    return 0
    
matrix['delta_price_lag'] = matrix.apply(select_trend, axis=1)
matrix['delta_price_lag'] = matrix['delta_price_lag'].astype(np.float16)
matrix['delta_price_lag'].fillna(0, inplace=True)


fetures_to_drop = ['item_avg_item_price', 'date_item_avg_item_price']
for i in lags:
    fetures_to_drop += ['date_item_avg_item_price_lag_'+str(i)]
    fetures_to_drop += ['delta_price_lag_'+str(i)]

matrix.drop(fetures_to_drop, axis=1, inplace=True)

In [ ]:
group = train.groupby(['date_block_num','shop_id']).agg({'revenue': ['sum']})
group.columns = ['date_shop_revenue']
group.reset_index(inplace=True)

matrix = pd.merge(matrix, group, on=['date_block_num','shop_id'], how='left')
matrix['date_shop_revenue'] = matrix['date_shop_revenue'].astype(np.float32)

group = group.groupby(['shop_id']).agg({'date_shop_revenue': ['mean']})
group.columns = ['shop_avg_revenue']
group.reset_index(inplace=True)

matrix = pd.merge(matrix, group, on=['shop_id'], how='left')
matrix['shop_avg_revenue'] = matrix['shop_avg_revenue'].astype(np.float32)

matrix['delta_revenue'] = (matrix['date_shop_revenue'] - matrix['shop_avg_revenue']) / matrix['shop_avg_revenue']
matrix['delta_revenue'] = matrix['delta_revenue'].astype(np.float16)

matrix = lag_feature(matrix, [1], 'delta_revenue')

matrix.drop(['date_shop_revenue','shop_avg_revenue','delta_revenue'], axis=1, inplace=True)

## 特定特征

In [ ]:
matrix['month'] = matrix['date_block_num'] % 12

In [ ]:
days = pd.Series([31,28,31,30,31,30,31,31,30,31,30,31])
matrix['days'] = matrix['month'].map(days).astype(np.int8)

In [ ]:
cache = {}
matrix['item_shop_last_sale'] = -1
matrix['item_shop_last_sale'] = matrix['item_shop_last_sale'].astype(np.int8)
for idx, row in matrix.iterrows():    
    key = str(row.item_id)+' '+str(row.shop_id)
    if key not in cache:
        if row.item_cnt_month!=0:
            cache[key] = row.date_block_num
    else:
        last_date_block_num = cache[key]
        matrix.at[idx, 'item_shop_last_sale'] = row.date_block_num - last_date_block_num
        cache[key] = row.date_block_num  

In [ ]:
cache = {}
matrix['item_last_sale'] = -1
matrix['item_last_sale'] = matrix['item_last_sale'].astype(np.int8)
for idx, row in matrix.iterrows():    
    key = row.item_id
    if key not in cache:
        if row.item_cnt_month!=0:
            cache[key] = row.date_block_num
    else:
        last_date_block_num = cache[key]
        if row.date_block_num>last_date_block_num:
            matrix.at[idx, 'item_last_sale'] = row.date_block_num - last_date_block_num
            cache[key] = row.date_block_num 

In [ ]:
matrix['item_shop_first_sale'] = matrix['date_block_num'] - matrix.groupby(['item_id','shop_id'])['date_block_num'].transform('min')
matrix['item_first_sale'] = matrix['date_block_num'] - matrix.groupby('item_id')['date_block_num'].transform('min')

## 最终的预测

选取13年以后的数据

In [ ]:
matrix = matrix[matrix['date_block_num'] > 11]

In [ ]:
def fill_na(df):
    for col in df.columns:
        if ('_lag_' in col) & (df[col].isnull().any()):
            if ('item_cnt' in col):
                df[col].fillna(0, inplace=True)         
    return df

matrix = fill_na(matrix)

In [ ]:
matrix.columns

In [ ]:
matrix.info()

In [ ]:
matrix.to_pickle('data.pkl')
del matrix
del cache
del group
del items
del shops
del cats
del train
# leave test for submission
gc.collect();

## 模型建立和训练

In [3]:
data = pd.read_pickle('data.pkl')

In [4]:
data = data[[
    'date_block_num',
    'shop_id',
    'item_id',
    'item_cnt_month',
    'city_code',
    'item_category_id',
    'type_code',
    'item_cnt_month_lag_1',
    'item_cnt_month_lag_2',
    'item_cnt_month_lag_3',
    'item_cnt_month_lag_5',
    'item_cnt_month_lag_12',
    'date_avg_item_cnt_lag_1',
    'date_item_avg_item_cnt_lag_1',
    'date_item_avg_item_cnt_lag_2',
    'date_item_avg_item_cnt_lag_3',
    'date_item_avg_item_cnt_lag_6',
    'date_item_avg_item_cnt_lag_12',
    'date_shop_avg_item_cnt_lag_1',
    'date_shop_avg_item_cnt_lag_2',
    'date_shop_avg_item_cnt_lag_3',
    'date_shop_avg_item_cnt_lag_6',
    'date_shop_avg_item_cnt_lag_12',
    'date_cat_avg_item_cnt_lag_1',
    'date_shop_cat_avg_item_cnt_lag_1',
    'date_shop_type_avg_item_cnt_lag_1',
    'date_shop_subtype_avg_item_cnt_lag_1',
    'date_city_avg_item_cnt_lag_1',
    'date_item_city_avg_item_cnt_lag_1',
    'date_type_avg_item_cnt_lag_1',
    'delta_price_lag',
    'month',
    'days',
    'item_shop_last_sale',
    'item_last_sale',    
    'item_shop_first_sale',
    'item_first_sale',
]]

数据划分

In [6]:
X_train = data[data.date_block_num < 33].drop(['item_cnt_month'], axis=1)
y_train = data[data.date_block_num < 33]['item_cnt_month']
X_valid = data[data.date_block_num == 33].drop(['item_cnt_month'],axis=1)
y_valid = data[data.date_block_num == 33]['item_cnt_month']
X_test = data[data.date_block_num == 34].drop(['item_cnt_month'], axis=1)

In [7]:
del data
gc.collect();

寻找重要特征

In [7]:
start = time.time()
rf = RandomForestRegressor()
rf.fit(X_train,y_train)
time.time()-start

1380.8614599704742

In [8]:
colname = X_train.columns
feature_import = rf.feature_importances_
feature_sort = []
for i,j in zip(colname,feature_import):
    feature_sort.append((i,j))
feature_sort = sorted(feature_sort,key=lambda x:x[1],reverse=True)
feature_import = map(lambda x:x[0],feature_sort)
feature_import = list(feature_import)
feature_import

['item_cnt_month_lag_1',
 'item_id',
 'date_item_avg_item_cnt_lag_1',
 'item_category_id',
 'date_shop_cat_avg_item_cnt_lag_1',
 'shop_id',
 'delta_price_lag',
 'month',
 'item_cnt_month_lag_3',
 'city_code',
 'date_block_num',
 'item_first_sale',
 'item_cnt_month_lag_2',
 'date_item_avg_item_cnt_lag_2',
 'date_shop_avg_item_cnt_lag_1',
 'date_item_city_avg_item_cnt_lag_1',
 'date_cat_avg_item_cnt_lag_1',
 'date_item_avg_item_cnt_lag_3',
 'date_shop_subtype_avg_item_cnt_lag_1',
 'date_city_avg_item_cnt_lag_1',
 'date_shop_type_avg_item_cnt_lag_1',
 'date_shop_avg_item_cnt_lag_2',
 'date_type_avg_item_cnt_lag_1',
 'date_item_avg_item_cnt_lag_6',
 'item_cnt_month_lag_5',
 'date_shop_avg_item_cnt_lag_3',
 'date_avg_item_cnt_lag_1',
 'type_code',
 'date_shop_avg_item_cnt_lag_6',
 'date_item_avg_item_cnt_lag_12',
 'days',
 'date_shop_avg_item_cnt_lag_12',
 'item_shop_first_sale',
 'item_cnt_month_lag_12',
 'item_shop_last_sale',
 'item_last_sale']

In [7]:
dump(feature_import,open('feature_import.pkl','wb'))

NameError: name 'feature_import' is not defined

In [8]:
feature_import = load(open('feature_import.pkl','rb'))

In [9]:
len(feature_import)

36

In [10]:
feature_import = feature_import[:30]
X_train = X_train[feature_import]
X_test = X_test[feature_import]

## 模型融合

level-1模型建立

In [13]:
rf_params = {
    'n_jobs': -1,
    'n_estimators': 100,
     #'warm_start': True, 
    'max_features': 0.2,
    #'max_features' : 'sqrt',
    #'max_depth': 6,
    'max_depth':2,
    'min_samples_leaf': 2,
    'verbose': 1
}

# Extra Trees Parameters
et_params = {
    'n_jobs': -1,
    #'n_estimators':500,
    'n_estimators':100,
    'max_features': 0.5,
    #'max_depth': 8,
    'max_depth':2,
    'min_samples_leaf': 2,
    'verbose': 1
}
'''
# AdaBoost parameters
ada_params = {
    'n_estimators': 500,
    'learning_rate' : 0.75
}
'''
# Gradient Boosting parameters
gbr_params = {
    #'n_estimators': 500,
    'n_estimators':100,
    'max_features': 0.2,
    #'max_depth': 5,
    'max_depth':2,
    'min_samples_leaf': 2,
    'verbose': 1
}

xgb_param = {
    'max_depth':8,
    'n_estimators':1000,
    'min_child_weight':300, 
    'colsample_bytree':0.8, 
    'subsample':0.8, 
    'eta':0.3,    
    'seed':42,
    'verbose':1,
    'early_stopping_rounds':10,
    'eval_metric':'rmse'
}

In [14]:
rf = RandomForestRegressor(**rf_params)
gbr = GradientBoostingRegressor(**gbr_params)
er = ExtraTreesRegressor(**et_params)

level-2模型建立

In [15]:
xgb = XGBRegressor(**xgb_param)

融合训练

In [16]:
stregr = StackingRegressor(regressors=[rf,er,gbr],
                          meta_regressor=xgb)

In [17]:
stregr.fit(X_train,y_train)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:  3.9min finished
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   47.6s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:  1.8min finished


      Iter       Train Loss   Remaining Time 
         1           1.3583            6.56m
         2           1.3119            5.65m
         3           1.2653            5.34m
         4           1.2194            5.39m
         5           1.1811            5.20m
         6           1.1373            5.07m
         7           1.0976            4.93m
         8           1.0644            4.79m
         9           1.0441            4.82m
        10           1.0268            4.83m
        20           0.9037            4.78m
        30           0.8598            3.87m
        40           0.8376            3.21m
        50           0.8290            2.64m
        60           0.8173            2.11m
        70           0.8122            1.56m
        80           0.8051            1.03m
        90           0.8017           30.71s
       100           0.7985            0.00s


[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    4.0s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:   10.1s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    4.4s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:   10.5s finished


[09:59:27] Tree method is automatically selected to be 'approx' for faster speed. To use old behavior (exact greedy algorithm on single machine), set tree_method to 'exact'.


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.


KeyboardInterrupt



In [ ]:
y_pred = stregr.predict(X_valid)

In [ ]:
y_test = stregr.predict(X_test).clip(0,20)

## 保存结果

In [ ]:
submission = pd.DataFrame({
    "ID": test.index, 
    "item_cnt_month": y_test
})
submission.to_csv('xgb_submission_5.csv', index=False)

In [ ]:
dump(stregr,open('ensemble_model.pkl','wb'))